In [7]:
from openai import OpenAI
import json
import requests 
import uuid
import time
import pandas as pd
import os
from dotenv import load_dotenv

In [ ]:
# .env파일 로드
load_dotenv()

# 환경변수에서 API 키 읽기
api_key = os.getenv("OPENAI_API_KEY")
# OpenAI 클라이언트 인스턴스 생성
client = OpenAI(api_key=api_key)

# Clova ocr 호출 

In [ ]:
def call_clova_ocr(file_path:str):
    api_url = ''
    secret_key = ''
    image_file = file_path

    request_json = {                                    # ocr 요청 json 데이터 구성
        'images': [
            {
                'format': 'jpg',
                'name': 'receipt'
            }
        ],
        'requestId': str(uuid.uuid4()),
        'version':'V2',
        'timestamp' :0
    }
    payload = {'message': json.dumps(request_json).encode('UTF-8')} # HTTP 요청 바디 준비
    files = {
    'file': open(image_file,'rb')} # 다른 코드(에러나면 고치기)
    headers = { 
        'X-OCR-SECRET': secret_key}
    response = requests.request("POST", api_url, headers=headers, data = payload, files = files)
    json_data = response.json() #응답 받은 데이터 .json으로 파싱해 딕셔너리 형태로 변환
    return json_data



# 응답 내용을 영수증 형태로 다시 변환
- images[0]: 보통 요청 하나에 이미지 하나만 처리하므로 첫 번째 이미지만 사용

- fields: 텍스트가 인식된 각 필드 정보 리스트 (인식된 글자, 위치 정보, 줄바꿈 여부 등 포함)

In [ ]:
def json_to_string(json_data: json):
    string_result= ''
    for i in json_data['images'][0]['fields']:
        if i['lineBreak'] == True: # 줄바꿈이 있으니까 줄바꿈 해라
            linebreak = '\n'
        else:
            linebreak = ' '
        string_result = string_result + i['inferText'] + linebreak
    return string_result

# 문자열 df로 변환
messages = [  
  {
    "role": "system",
    "content": "당신은 영수증을 분석하고 JSON 형식으로 정확히 구조화하는 유용한 어시스턴트입니다. "
               "결과는 다음과 같은 JSON 형식을 따라야 합니다: "  
               "{\"date\": \"yyyy/mm/dd\", \"items\": [{\"item_name\": \"항목 이름\", \"price\": 100, \"quantity\": 3, \"amount\": 300}]} "  
               "항목이 무료인 경우 price는 0으로 설정하고, quantity만 반영하십시오. "  
               "price는 개별 항목당 가격이며, amount는 price × quantity입니다. "  
               "항목명에 '할인'이라는 단어가 포함되어 있다면 할인 항목으로 처리해야 합니다. "  
               "면세 물품은 총합 계산에서 제외해야 합니다. "  
               "단일 항목만 있는 영수증의 경우, 그 항목의 price는 전체 금액과 같아야 합니다."  
  },    
  {  
    "role": "user",  
    "content": f"다음 영수증을 분석해주세요:\n{string_result}\n"  
               "항목 내 할인 또는 음수 금액이 있다면, 이를 최종 합계에 정확히 반영해주세요."  
  }
]

In [ ]:
# 문자열을 데이터 프레임으로 변환
def string_to_xlsx(string_result: str, receipt_num: int):
    client = OpenAI()

    # ChatGPT 호출
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant to analyze the receipt. "
                           "Please ensure that the receipt's data is correctly formatted in the JSON format as shown in this example: "
                           "{\"date\": \"yyyy/mm/dd\", \"items\": [{\"item_name\": \"Item_1\", \"price\": 100, \"quantity\": 3, \"amount\": 300}]}. "
                           "If an item is free, set its price to 0 and only count quantity."
                           "Be aware that 'price' refers to the cost per item, while 'amount' refers to the total cost of an item line (price multiplied by quantity)."
                           "Pay special attention to items that include the word '할인' in their item_name, as these represent discounts. "
                           "'면세 물품' items should not be included in the total calculation."
                           "For single-item purchases, ensure that the item's price reflects the receipt's final total."
            }
            ,
            {
                "role": "user",
                "content": f"Please analyze this receipt: \n{string_result}\n "
                           "Make sure to reflect any discounts or negative amounts in the final total."
            }

        ]
    )
    # ChatGPT 회신
    message = response.choices[0].message.content 
    print(response.usage) #GPT API 호출에서 토큰 사용량 출력 (prompt, completion 포함)
    print(message) 

    # JSON 문자열을 파이썬 딕셔너리로 변환
    data = json.loads(message) 

    # pandas DataFrame으로 변환 및 날짜 추가
    df = pd.DataFrame(data['items'])
    df['date'] = data['date']
    df['receipt_num'] = receipt_num

    return df




In [ ]:

# 영수증의 파일 경로 리스트
image_folder= r"C:\python_haea\영수증_모음"
file_list=[]
for file_name in os.listdir(image_folder):
    if file_name.lower().endswith(('.jpg','jpeg','.png')):
        file_path = os.path.join(image_folder, file_name)
        file_list.append(file_path)

# 데이터 프레임 생성
df_one = pd.DataFrame()

# 영수증 번호
receipt_num = 0

# 반복문을 이용한 영수증 정리
for i in file_list:
    receipt_num = receipt_num + 1
    receipt_json = call_clova_ocr(i)
    receipt_string = json_to_string(receipt_json)
    receipt_pd = string_to_xlsx(receipt_string, receipt_num)
    df_one = pd.concat([df_one, receipt_pd], ignore_index=True)

# 정리내용 확인용
print(df_one)

# 엑셀 파일로 저장.

df_one.to_excel(r"C:\python_haea\영수증_자동입력결과.xlsx", index=False)
print(f"엑셀 파일이 저장 되었습니다.")

CompletionUsage(completion_tokens=234, prompt_tokens=657, total_tokens=891, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
{
  "date": "2025/06/27",
  "items": [
    {"item_name": "레몬하이블", "price": 5500, "quantity": 1, "amount": 5500},
    {"item_name": "자몽하이블", "price": 5500, "quantity": 2, "amount": 11000},
    {"item_name": "셀프김치주먹밥", "price": 5000, "quantity": 1, "amount": 5000},
    {"item_name": "오돌뼈볶음과주먹", "price": 25000, "quantity": 1, "amount": 25000},
    {"item_name": "처음처럼", "price": 5000, "quantity": 1, "amount": 5000},
    {"item_name": "음료", "price": 2000, "quantity": 1, "amount": 2000}
  ]
}
CompletionUsage(completion_tokens=276, prompt_tokens=647, total_tokens=923, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_predicti

In [11]:
df_one

,item_name,price,quantity,amount,date,receipt_num
0,레몬하이블,5500,1,5500,2025/06/27,1
1,자몽하이블,5500,2,11000,2025/06/27,1
2,셀프김치주먹밥,5000,1,5000,2025/06/27,1
3,오돌뼈볶음과주먹,25000,1,25000,2025/06/27,1
4,처음처럼,5000,1,5000,2025/06/27,1
5,음료,2000,1,2000,2025/06/27,1
6,레몬하이블,5500,1,5500,2025/06/27,2
7,자몽하이블,5500,2,11000,2025/06/27,2
8,셀프김치주먹밥,5000,1,5000,2025/06/27,2
9,오돌뼈볶음과주먹,25000,1,25000,2025/06/27,2
